In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets, transforms

In [2]:
class RNN(nn.Module):
  def __init__(self):
    super().__init__()
    
  def forward(self, x, state):
    return x + state
  
p = 99

x = Variable(torch.Tensor([
  [1, 1, 1, p, p],
  [1, 1, p, p, p],
  [1, p, p, p, p]
]), requires_grad=True)

state = Variable(torch.zeros(3))
rnn = RNN()
for t in range(x.size()[1]):
  current = x[:, t]
  valid = (current != p).float()
  state = rnn(current, state) * valid.float() + state * (1 - valid)
  
print(state)
state.mean().backward()
print(x.grad)

fail

Variable containing:
 3
 2
 1
[torch.FloatTensor of size 3]

Variable containing:
 0.3333  0.3333  0.3333  0.0000  0.0000
 0.3333  0.3333  0.0000  0.0000  0.0000
 0.3333  0.0000  0.0000  0.0000  0.0000
[torch.FloatTensor of size 3x5]



NameError: name 'fail' is not defined

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/Datasets/mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/Datasets/mnist', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1024, shuffle=True)

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 3)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    # Max pooling over a (2, 2) window
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    # If the size is a square you can only specify a single number
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    x = F.log_softmax(x, dim=1)
    return x

  def num_flat_features(self, x):
    size = x.size()[1:]  # all dimensions except the batch dimension
    num_features = 1
    for s in size:
        num_features *= s
    return num_features

model = Net()
print(model)

In [ ]:
epochs = 10
learning_rate = 0.001

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(1, epochs + 1):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    
  model.eval()
  test_loss = 0
  correct = 0
  for batch_idx, (data, target) in enumerate(test_loader):
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  test_loss /= len(test_loader.dataset)
  print('epoch: {}, loss: {:.4f}, accuracy: {:.2f}%'.format(
      epoch, test_loss, 100. * correct / len(test_loader.dataset)))

In [ ]:
print(loss.grad_fn) 
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])